In [4]:
import os
import pandas as pd
import numpy as np
import json
from config import * 
import time

data_dir = os.path.join(os.getcwd(), 'data')
datapath = os.path.join(data_dir, 'parking-meters.csv')

In [5]:


df = pd.read_csv(datapath, sep=';')
print(df.shape)
# df = df.loc[~df.METERHEAD.isin(['Single / Disability', 'Single Motorbike', 'Twin / Disability'])]
df = df.loc[df.METERHEAD.isin(['Single', 'Twin'])]
df

(7954, 21)


,METERHEAD,R_MF_9A_6P,R_MF_6P_10,R_SA_9A_6P,R_SA_6P_10,R_SU_9A_6P,R_SU_6P_10,RATE_MISC,TIMEINEFFE,T_MF_9A_6P,...,T_SA_9A_6P,T_SA_6P_10,T_SU_9A_6P,T_SU_6P_10,TIME_MISC,CREDITCARD,PAY_PHONE,Geom,Geo Local Area,METERID
0,Twin,$2.00,$2.00,$2.00,$2.00,$2.00,$2.00,NaN,METER IN EFFECT: 9:00 AM TO 10:00 PM,2 Hr,...,2 Hr,4 Hr,2 Hr,4 Hr,NaN,No,50584,"{""coordinates"": [-123.14344688271457, 49.26438...",Fairview,902403
1,Twin,$1.00,$1.00,$1.00,$1.00,$1.00,$1.00,NaN,METER IN EFFECT: 9:00 AM TO 10:00 PM,2 Hr,...,2 Hr,4 Hr,2 Hr,4 Hr,NaN,No,66072,"{""coordinates"": [-123.12523294533561, 49.28183...",West End,640903
2,Twin,$1.00,$1.00,$1.00,$1.00,$1.00,$1.00,NaN,METER IN EFFECT: 9:00 AM TO 10:00 PM,2 Hr,...,2 Hr,4 Hr,2 Hr,4 Hr,NaN,No,66073,"{""coordinates"": [-123.12533781552285, 49.28176...",West End,640907
3,Single,$1.00,$1.00,$1.00,$1.00,$1.00,$1.00,NaN,METER IN EFFECT: 9:00 AM TO 10:00 PM,2 Hr,...,2 Hr,4 Hr,2 Hr,4 Hr,NaN,No,66055,"{""coordinates"": [-123.12590142819074, 49.28138...",West End,640925
4,Twin,$1.00,$1.00,$1.00,$1.00,$1.00,$1.00,NaN,METER IN EFFECT: 9:00 AM TO 10:00 PM,2 Hr,...,2 Hr,4 Hr,2 Hr,4 Hr,NaN,No,60163,"{""coordinates"": [-123.1248713052417, 49.281799...",Downtown,640914
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7949,Twin,$2.00,$1.00,$2.00,$1.00,$2.00,$1.00,NaN,METER IN EFFECT: 9:00 AM TO 10:00 PM,2 Hr,...,2 Hr,4 Hr,2 Hr,4 Hr,NaN,No,54869,"{""coordinates"": [-123.10107548622334, 49.24664...",Riley Park,514224
7950,Twin,$2.00,$1.00,$2.00,$1.00,$2.00,$1.00,NaN,METER IN EFFECT: 9:00 AM TO 10:00 PM,2 Hr,...,2 Hr,4 Hr,2 Hr,4 Hr,NaN,No,54974,"{""coordinates"": [-123.1012601221966, 49.247816...",Riley Park,514203
7951,Twin,$2.00,$1.00,$2.00,$1.00,$2.00,$1.00,NaN,METER IN EFFECT: 9:00 AM TO 10:00 PM,2 Hr,...,2 Hr,4 Hr,2 Hr,4 Hr,NaN,No,54919,"{""coordinates"": [-123.10128955730812, 49.24747...",Riley Park,514215
7952,Twin,$1.00,$1.00,$1.00,$1.00,$1.00,$1.00,NaN,METER IN EFFECT: 9:00 AM TO 10:00 PM,2 Hr,...,2 Hr,4 Hr,2 Hr,4 Hr,NaN,No,56701,"{""coordinates"": [-123.18728854800088, 49.26357...",West Point Grey,D03729


In [6]:


def get_latln(row):
    geom = json.loads(row)
    return geom['coordinates'][1], geom['coordinates'][0]

df = df.loc[~df.Geom.isna()]
df['latln'] = df['Geom'].apply(get_latln)
#Find rest
df_no_twins = df.loc[~(df.METERHEAD=='Twin')]
df_no_twins.reset_index()
#Find all twins
twins = df.loc[df.METERHEAD=='Twin']
twins.reset_index(inplace=True)
print(twins.index)
#Make sure we are finding unique spots
seen_twins = set()
for i, row in twins.iterrows():
    #find the match
    pair = tuple(np.sort(np.where(twins.latln == row['latln'])[0]))
    if len(pair) != 2:
        continue
    if pair not in seen_twins:
        assert i == pair[0] #Current one is the first
        seen_twins.add(pair)
    else:
        continue
    # print(pair)
    check_columns = ['METERHEAD', 'Geo Local Area', 'latln']
    meter1, meter2 = twins.loc[pair[0]], twins.loc[pair[1]]
    for col in check_columns:
        # print(col)
        # print(meter1[col])
        # print(meter2[col])
        v1, v2 = meter1[col], meter2[col]
        if not pd.isnull(v1) and not pd.isnull(v2):
            assert v1 == v2
        # assert meter1['Geo Local Area'] == meter2['Geo Local Area']
    # print(pair)
    # print(twins.loc[pair[0]].METERID)




print(len(twins))
print(len(twins)//2)
print(100*len(seen_twins)/len(twins))

RangeIndex(start=0, stop=6086, step=1)


/var/folders/52/md6yz3dn2g344wf8pqqb7kc80000gn/T/ipykernel_10177/947984805.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['latln'] = df['Geom'].apply(get_latln)


6086
3043
49.90141307919816


In [9]:

    
'''
Use Twins
for each pair, use first one's info to create a double spot
'''
merged_twins = []
for pair in seen_twins:
    first, second = pair
    first_meter = twins.loc[first]
    merged_meter = first_meter
    merged_meter['METERID'] = (twins.loc[first]['METERID'], twins.loc[second]['METERID'])
    merged_twins.append(first_meter)
merged_twins = pd.DataFrame(merged_twins)
merged_twins.reset_index(inplace=True)

/var/folders/52/md6yz3dn2g344wf8pqqb7kc80000gn/T/ipykernel_10177/3267042566.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_meter['METERID'] = (twins.loc[first]['METERID'], twins.loc[second]['METERID'])


,level_0,index,METERHEAD,R_MF_9A_6P,R_MF_6P_10,R_SA_9A_6P,R_SA_6P_10,R_SU_9A_6P,R_SU_6P_10,RATE_MISC,...,T_SA_6P_10,T_SU_9A_6P,T_SU_6P_10,TIME_MISC,CREDITCARD,PAY_PHONE,Geom,Geo Local Area,METERID,latln
0,4132,5508,Twin,$5.00,$5.00,$5.00,$5.00,$5.00,$5.00,NaN,...,4 Hr,2 Hr,4 Hr,NaN,No,51224,"{""coordinates"": [-123.1154915217198, 49.264085...",Fairview,"(B80508, B80510)","(49.26408588041858, -123.1154915217198)"
1,3923,5249,Twin,$2.00,$2.00,$2.00,$2.00,$2.00,$2.00,NaN,...,4 Hr,3 Hr,4 Hr,NaN,No,54587,"{""coordinates"": [-123.15536308356153, 49.23773...",Arbutus-Ridge,"(935316, 935314)","(49.23773425581723, -123.15536308356153)"
2,2518,3361,Twin,$1.00,$1.00,$1.00,$1.00,$1.00,$1.00,NaN,...,4 Hr,2 Hr,4 Hr,NaN,No,53787,"{""coordinates"": [-123.210137691826, 49.2637691...",West Point Grey,"(D04518, D04520)","(49.263769133101064, -123.210137691826)"
3,33,39,Twin,$1.00,$1.00,$1.00,$1.00,$1.00,$1.00,NaN,...,4 Hr,3 Hr,4 Hr,NaN,No,66134,"{""coordinates"": [-123.12384806563952, 49.27324...",Downtown,"(321302, 321304)","(49.273243359913664, -123.12384806563952)"
4,49,56,Twin,$1.00,$4.00,$1.00,$4.00,$1.00,$4.00,NaN,...,4 Hr,2 Hr,4 Hr,NaN,No,51575,"{""coordinates"": [-123.06959037014099, 49.26469...",Grandview-Woodland,"(6C2210, 6C2208)","(49.26469493711841, -123.06959037014099)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3032,1676,2216,Twin,$2.00,$1.00,$2.00,$1.00,$2.00,$1.00,NaN,...,4 Hr,2 Hr,4 Hr,NaN,No,50098,"{""coordinates"": [-123.13403515397614, 49.26343...",Fairview,"(301302, 301304)","(49.263437973203764, -123.13403515397614)"
3033,10,13,Twin,$4.00,$6.00,$4.00,$6.00,$4.00,$6.00,NaN,...,4 Hr,2 Hr,4 Hr,NaN,No,64513,"{""coordinates"": [-123.12586490354587, 49.28519...",West End,"(121114, 121116)","(49.28519388910239, -123.12586490354587)"
3034,3074,4054,Twin,$3.00,$2.00,$3.00,$2.00,$3.00,$2.00,NaN,...,4 Hr,2 Hr,4 Hr,NaN,No,62186,"{""coordinates"": [-123.11455435861743, 49.27786...",Downtown,"(120110, 120112)","(49.277863646882906, -123.11455435861743)"
3035,69,80,Twin,$2.00,$1.00,$2.00,$1.00,$2.00,$1.00,NaN,...,4 Hr,2 Hr,4 Hr,NaN,No,54699,"{""coordinates"": [-123.1607609351604, 49.234584...",Kerrisdale,"(K12314, K12316)","(49.23458422626698, -123.1607609351604)"


In [13]:
pd.concat([df_no_twins, merged_twins])

,METERHEAD,R_MF_9A_6P,R_MF_6P_10,R_SA_9A_6P,R_SA_6P_10,R_SU_9A_6P,R_SU_6P_10,RATE_MISC,TIMEINEFFE,T_MF_9A_6P,...,T_SU_6P_10,TIME_MISC,CREDITCARD,PAY_PHONE,Geom,Geo Local Area,METERID,latln,level_0,index
3,Single,$1.00,$1.00,$1.00,$1.00,$1.00,$1.00,NaN,METER IN EFFECT: 9:00 AM TO 10:00 PM,2 Hr,...,4 Hr,NaN,No,66055,"{""coordinates"": [-123.12590142819074, 49.28138...",West End,640925,"(49.2813874505898, -123.12590142819074)",NaN,NaN
5,Single,$5.00,$3.00,$5.00,$3.00,$5.00,$3.00,NaN,METER IN EFFECT: 9:00 AM TO 10:00 PM,3 Hr,...,4 Hr,NaN,No,66859,"{""coordinates"": [-123.1208487679332, 49.273572...",Downtown,160120,"(49.27357210772267, -123.1208487679332)",NaN,NaN
30,Single,$1.00,$1.00,$1.00,$1.00,$1.00,$1.00,NaN,METER IN EFFECT: 9:00 AM TO 10:00 PM,3 Hr,...,4 Hr,NaN,No,61288,"{""coordinates"": [-123.12297806641084, 49.27807...",Downtown,601023,"(49.278070423337454, -123.12297806641084)",NaN,NaN
48,Single,$1.00,$1.00,$1.00,$1.00,$1.00,$1.00,NaN,METER IN EFFECT: 9:00 AM TO 10:00 PM,2 Hr,...,4 Hr,NaN,No,53777,"{""coordinates"": [-123.21091545808456, 49.26378...",West Point Grey,D04534,"(49.2637811457829, -123.21091545808456)",NaN,NaN
63,Single,$4.00,$1.00,$4.00,$1.00,$4.00,$1.00,NaN,METER IN EFFECT: 9:00 AM TO 10:00 PM,2 Hr,...,4 Hr,NaN,No,54307,"{""coordinates"": [-123.10791776294424, 49.26660...",Mount Pleasant,B50121,"(49.26660058864532, -123.10791776294424)",NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3032,Twin,$2.00,$1.00,$2.00,$1.00,$2.00,$1.00,NaN,METER IN EFFECT: 9:00 AM TO 10:00 PM,2 Hr,...,4 Hr,NaN,No,50098,"{""coordinates"": [-123.13403515397614, 49.26343...",Fairview,"(301302, 301304)","(49.263437973203764, -123.13403515397614)",1676.0,2216.0
3033,Twin,$4.00,$6.00,$4.00,$6.00,$4.00,$6.00,NaN,METER IN EFFECT: 9:00 AM TO 10:00 PM,2 Hr,...,4 Hr,NaN,No,64513,"{""coordinates"": [-123.12586490354587, 49.28519...",West End,"(121114, 121116)","(49.28519388910239, -123.12586490354587)",10.0,13.0
3034,Twin,$3.00,$2.00,$3.00,$2.00,$3.00,$2.00,NaN,METER IN EFFECT: 9:00 AM TO 10:00 PM,2 Hr,...,4 Hr,NaN,No,62186,"{""coordinates"": [-123.11455435861743, 49.27786...",Downtown,"(120110, 120112)","(49.277863646882906, -123.11455435861743)",3074.0,4054.0
3035,Twin,$2.00,$1.00,$2.00,$1.00,$2.00,$1.00,NaN,METER IN EFFECT: 9:00 AM TO 10:00 PM,2 Hr,...,4 Hr,NaN,No,54699,"{""coordinates"": [-123.1607609351604, 49.234584...",Kerrisdale,"(K12314, K12316)","(49.23458422626698, -123.1607609351604)",69.0,80.0
